In [44]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver
import numpy as np

## Pull Each Team's Roster URL

In [96]:
team_url = 'https://web.archive.org/web/20181202093151/http://www.espn.com/nfl/players'
request = requests.get(team_url)
soup = BeautifulSoup(request.text, 'html.parser')

In [97]:
["https://web.archive.org" + x["href"] for x in soup.findAll('a') if ("team/" in x["href"]) and ("roster/" in x["href"])]

['https://web.archive.org/web/20181202093151/http://www.espn.com/nfl/team/roster/_/name/buf/buffalo-bills',
 'https://web.archive.org/web/20181202093151/http://www.espn.com/nfl/team/roster/_/name/mia/miami-dolphins',
 'https://web.archive.org/web/20181202093151/http://www.espn.com/nfl/team/roster/_/name/ne/new-england-patriots',
 'https://web.archive.org/web/20181202093151/http://www.espn.com/nfl/team/roster/_/name/nyj/new-york-jets',
 'https://web.archive.org/web/20181202093151/http://www.espn.com/nfl/team/roster/_/name/dal/dallas-cowboys',
 'https://web.archive.org/web/20181202093151/http://www.espn.com/nfl/team/roster/_/name/nyg/new-york-giants',
 'https://web.archive.org/web/20181202093151/http://www.espn.com/nfl/team/roster/_/name/phi/philadelphia-eagles',
 'https://web.archive.org/web/20181202093151/http://www.espn.com/nfl/team/roster/_/name/wsh/washington-redskins',
 'https://web.archive.org/web/20181202093151/http://www.espn.com/nfl/team/roster/_/name/den/denver-broncos',
 'htt

In [98]:
teams = ["https://web.archive.org" + x["href"] for x in soup.findAll('a') if ("team/" in x["href"]) and ("roster/" in x["href"])]

In [100]:
[x.text for x in soup.findAll('div', class_ = "mod-header stathead")]

['AFC East',
 'NFC East',
 'AFC West',
 'NFC West',
 'AFC North',
 'NFC North',
 'AFC South',
 'NFC South']

# Getting Player Ids

In [119]:
teams[0].split("/name/")[1].split("/")[0].upper()

'BUF'

In [120]:
playerDf_tot = pd.DataFrame()

In [121]:
for team_url in teams:
    
    print("Pulling Players From " + team_url.split("/name/")[1].split("/")[0].upper())

    request = requests.get(team_url)
    soup = BeautifulSoup(request.text, 'html.parser')

    player_urls = [x for x in [x['href'] for x in soup.findAll('a') if 'href' in str(x)] if ("player/" in x)]

    player_ids = [x.split("/id/")[1] for x in player_urls]

    player_image_urls = ["https://a.espncdn.com/combiner/i?img=/i/headshots/nfl/players/full/" + 
                        x + ".png&w=350&h=254" for x in player_ids]

    player_names = np.array([x.text for x in soup.findAll('a') if 'href' in str(x)])[np.array([("player/" in x) for x in [x['href'] for x in soup.findAll('a') if 'href' in str(x)]])]

    player_positions = [x.text for x in soup.findAll('td', class_ = 'Table2__td')][2::8]

    player_schools = [x.text for x in soup.findAll('td', class_ = 'Table2__td')][7::8]

    playerDf = pd.DataFrame({"displayName": player_names,
                  "team": team_url.split("/name/")[1].split("/")[0].upper(),
                  "position": player_positions,
                  "collegeName": player_schools,
                  "playerId":player_ids,
                  "playerImageUrl":player_image_urls})
    
    playerDf_tot = pd.concat([playerDf_tot,
                              playerDf])

Pulling Players From BUF
Pulling Players From MIA
Pulling Players From NE
Pulling Players From NYJ
Pulling Players From DAL
Pulling Players From NYG
Pulling Players From PHI
Pulling Players From WSH
Pulling Players From DEN
Pulling Players From KC
Pulling Players From LAC
Pulling Players From OAK
Pulling Players From ARI
Pulling Players From LAR
Pulling Players From SF
Pulling Players From SEA
Pulling Players From BAL
Pulling Players From CIN
Pulling Players From CLE
Pulling Players From PIT
Pulling Players From CHI
Pulling Players From DET
Pulling Players From GB
Pulling Players From MIN
Pulling Players From HOU
Pulling Players From IND
Pulling Players From JAX
Pulling Players From TEN
Pulling Players From ATL
Pulling Players From CAR
Pulling Players From NO
Pulling Players From TB


In [122]:
56*32

1792

In [127]:
playerDf_tot

,displayName,team,position,collegeName,playerId,playerImageUrl
0,Josh Allen,BUF,QB,Wyoming,3918298,https://a.espncdn.com/combiner/i?img=/i/headsh...
1,Derek Anderson,BUF,QB,Oregon State,8627,https://a.espncdn.com/combiner/i?img=/i/headsh...
2,Matt Barkley,BUF,QB,USC,15948,https://a.espncdn.com/combiner/i?img=/i/headsh...
3,Chris Ivory,BUF,RB,Tiffin University,13587,https://a.espncdn.com/combiner/i?img=/i/headsh...
4,Taiwan Jones,BUF,RB,Eastern Washington,14167,https://a.espncdn.com/combiner/i?img=/i/headsh...
...,...,...,...,...,...,...
59,Isaiah Johnson,TB,S,South Carolina,2570484,https://a.espncdn.com/combiner/i?img=/i/headsh...
60,Jordan Whitehead,TB,S,Pittsburgh,3895798,https://a.espncdn.com/combiner/i?img=/i/headsh...
61,Cairo Santos,TB,PK,Tulane,17427,https://a.espncdn.com/combiner/i?img=/i/headsh...
62,Bryan Anger,TB,P,California,14950,https://a.espncdn.com/combiner/i?img=/i/headsh...


In [128]:
playerDf_tot.to_csv("/Users/James/Desktop/CoverageNet/src/00_data_wrangle/helper_tables/playerImages.csv",
                 index = False)

# Scraping College Images

In [129]:
team_url = 'https://www.espn.com/college-football/teams'
request = requests.get(team_url)
soup = BeautifulSoup(request.text, 'html.parser')

In [149]:
college_full_names = [x["title"] for x in soup.findAll('img', class_ = "Image Logo Logo__lg")]

In [163]:
college_team_links = ['https://espn.com' + x.find('a')["href"] for x in soup.findAll('div', class_ = "TeamLinks__Links")]

In [152]:
college_team_ids = [x.find('a')["href"].split('id/')[1] for x in soup.findAll('div', class_ = "TeamLinks__Links")]

In [154]:
college_team_images = ["https://a.espncdn.com/combiner/i?img=/i/teamlogos/ncaa/500/" +\
                        x + ".png&h=80&w=80&scale=crop&cquality=40&location=origin" for x in college_team_ids]

In [170]:
# gettng college name
college_name = []
for link in college_team_links:
    team_url = link
    request = requests.get(team_url)
    soup = BeautifulSoup(request.text, 'html.parser')
    name = soup.find('span', class_ = "db pr3 nowrap fw-bold").text
    college_name.append(name)
    print(name)

Cincinnati
East Carolina
Houston
Memphis
Navy
SMU
South Florida
Temple
Tulane
Tulsa
UCF
Boston College
Clemson
Duke
Florida State
Georgia Tech
Louisville
Miami
NC State
North Carolina
Notre Dame
Pittsburgh
Syracuse
Virginia
Virginia Tech
Wake Forest
Baylor
Iowa State
Kansas
Kansas State
Oklahoma
Oklahoma State
TCU
Texas
Texas Tech
West Virginia
Illinois
Indiana
Iowa
Maryland
Michigan State
Michigan
Minnesota
Nebraska
Northwestern
Ohio State
Penn State
Purdue
Rutgers
Wisconsin
Charlotte
Florida Atlantic
Florida International
Louisiana Tech
Marshall
Middle Tennessee
North Texas
Old Dominion
Rice
Southern Mississippi
UAB
UTEP
UTSA
Western Kentucky
Army
BYU
Liberty
New Mexico State
UConn
UMass
Akron
Ball State
Bowling Green
Buffalo
Central Michigan
Eastern Michigan
Kent State
Miami (OH)
Northern Illinois
Ohio
Toledo
Western Michigan
Air Force
Boise State
Colorado State
Fresno State
Hawai'i
Nevada
New Mexico
San Diego State
San José State
UNLV
Utah State
Wyoming
Arizona State
Arizona
Califo

In [172]:
collegeTeamsDf = pd.DataFrame({"collegeFullName": college_full_names,
                              "collegeName": college_name,
                              "collegeId": college_team_ids,
                              "collegeImageUrl": college_team_images})

In [173]:
collegeTeamsDf

,collegeFullName,collegeName,collegeId,collegeImageUrl
0,Cincinnati Bearcats,Cincinnati,2132,https://a.espncdn.com/combiner/i?img=/i/teamlo...
1,East Carolina Pirates,East Carolina,151,https://a.espncdn.com/combiner/i?img=/i/teamlo...
2,Houston Cougars,Houston,248,https://a.espncdn.com/combiner/i?img=/i/teamlo...
3,Memphis Tigers,Memphis,235,https://a.espncdn.com/combiner/i?img=/i/teamlo...
4,Navy Midshipmen,Navy,2426,https://a.espncdn.com/combiner/i?img=/i/teamlo...
...,...,...,...,...
125,Louisiana Ragin' Cajuns,Louisiana,309,https://a.espncdn.com/combiner/i?img=/i/teamlo...
126,South Alabama Jaguars,South Alabama,6,https://a.espncdn.com/combiner/i?img=/i/teamlo...
127,Texas State Bobcats,Texas State,326,https://a.espncdn.com/combiner/i?img=/i/teamlo...
128,Troy Trojans,Troy,2653,https://a.espncdn.com/combiner/i?img=/i/teamlo...


In [176]:
collegeTeamsDf.to_csv("/Users/James/Desktop/CoverageNet/src/00_data_wrangle/helper_tables/dashboard_college_info.csv",
                 index = False)